In [6]:
from datetime import datetime
datetime.now().isoformat()

'2025-06-02T01:30:28.677271'

In [ ]:
# pip install langchain-ollama ollama
from langchain_ollama import OllamaLLM
from langchain_ollama import ChatOllama


In [ ]:

model = ChatOllama(model = "llama3.2:1b", streaming = True)

response = model.invoke(input="What is a car in one line")
print("response",response)

In [ ]:
# from langchain.chat_models import ChatOllama
# from langchain_community.chat_models import ChatOllama
import json
from langchain.schema import SystemMessage, HumanMessage
from langchain_ollama import ChatOllama
from langchain.agents import Tool, initialize_agent
from langchain.agents.agent_types import AgentType

def classify_intent(user_input: str) -> str:
    try:
        llm = ChatOllama(model="llama3.2:1b")
        messages = [
            SystemMessage(
                content=(
                    "You are an AI assistant that classifies the intent of a user message into one of the following:\n"
                    "- 'chitchat': Small talk or unrelated to appointments\n"
                    "- 'schedule': Request to book a new appointment\n"
                    "- 'modify': Request to reschedule or cancel an existing appointment\n\n"
                    "Respond ONLY with one of these labels: chitchat, schedule, modify."
                    "STRICTLY Respond in JSON format ONLY like this:\n"
                    '{ "intent": "schedule" }'
                )
            ),
            HumanMessage(content=user_input)
        ]
        response = llm(messages)
        intent = json.loads(response.content.strip())["intent"]
        # intent = response.content.strip().lower()

        if intent not in {"chitchat", "schedule", "modify"}:
            return "chitchat"
        return intent
    except Exception as e:
        print(e)
        return "chitchat"

intent_tool = Tool(
    name="IntentClassifier",
    func=classify_intent,
    description="Classifies user input into: chitchat, schedule, or modify"
)

def create_intent_agent():
    llm = ChatOllama(model="llama3.2:1b")
    agent = initialize_agent(
        tools=[intent_tool],
        llm=llm,
        agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
        verbose=True,
        handle_parsing_errors=True 
    )
    return agent


In [ ]:
if __name__ == "__main__":
    agent = create_intent_agent()
    response = agent.run("Hey, can I reschedule my meeting to tomorrow afternoon?")
    print(f"Intent detected: {response}")

# Intent

In [1]:
from langchain_ollama import ChatOllama
from langchain.schema import SystemMessage, HumanMessage
import json
import re

llm = ChatOllama(model="gemma3:4b")

def detect_intent(user_input: str) -> str:
    messages = [
        SystemMessage(
            content=(
                "You are an intent classifier.\n"
                "Classify the user input STRICTLY into ONLY ONE of the following categories:\n"
                "1. chitchat\n"
                "2. schedule\n"
                "3. modify\n\n"
                "STRICTLY Respond only in this JSON format:\n"
                '{ "intent": "<chitchat or schedule or modify>"}'
                "DO NOT BLUFF"
            )
        ),
        HumanMessage(content=user_input)
    ]

    response = llm(messages)
    content = response.content.strip()
    content = re.sub(r"```json|```", "", content).strip()
    try:
        match = re.search(r'\{.*?\}', content, re.DOTALL)
        if match:
            json_str = match.group(0).replace("'", '"')
            parsed = json.loads(json_str)
            intent = parsed.get("intent", "chitchat")
            if str(intent).lower() in ["chitchat", "schedule", "modify"]:
                return intent
    except Exception as e:
        print("Intent parsing failed:", e)

    print("Raw model output:", response.content)
    return "chitchat"


In [16]:
if __name__ == "__main__":
    print(detect_intent("Hey, can I reschedule my appointment to next Monday?"))  # -> modify
    print(detect_intent("I want to book a new appointment for Tuesday"))          # -> schedule
    print(detect_intent("How are you today?"))                                    # -> chitchat
    print(detect_intent("Can you move my appointment with Dr. Smith from June 10 at 2 PM to June 12 at 3:30 PM?")) # -> modify

modify
schedule
chitchat
modify


# Schedule

In [3]:
import sys
sys.path.append(r"C:\Users\unmes\Documents\RAGful_dev\meet_scheduler")
from models.event import init_event_db,fetch_all_events
init_event_db()
fetch_all_events()

[]

In [11]:
import sys
sys.path.append(r"C:\Users\unmes\Documents\RAGful_dev\meet_scheduler")

from langchain_ollama import ChatOllama
from langchain.prompts import PromptTemplate
from langchain.schema import HumanMessage, SystemMessage
from datetime import datetime, timedelta
import sqlite3
import json
import re
from models.event import insert_event

# Initialize the LLM
llm = ChatOllama(model="gemma3:4b") 

# Prompt Template
SCHEDULE_PROMPT = """
You are an intelligent meeting assistant.

Extract the following meeting details from the user query:
- participant: list of people involved
- date: YYYY-MM-DD
- time: HH:MM in 24-hour format
- type: one of [consultation, followup, meeting, sales, support, emergency, other]
- purpose: max 50 characters
- duration_minutes: integer

Respond only in JSON format like this:
{{
  "participant": ["Dr. Smith"],
  "date": "2025-06-10",
  "time": "14:00",
  "type": "consultation",
  "purpose": "Health check-up",
  "duration_minutes": 30
}}

User: {user_input}
"""

def clean_llm_response(raw_response):
    try:
        raw_response = re.sub(r"```json|```", "", raw_response).strip()
        raw_response = raw_response.replace("True", "true").replace("False", "false")
        match = re.search(r'\{.*?\}', raw_response, re.DOTALL)
        if match:
            return json.loads(match.group(0))
    except Exception as e:
        print("JSON parse error:", e)
    return None

def is_slot_available(date, time, duration):
    conn = sqlite3.connect("calendar_events.db")
    cursor = conn.cursor()

    start_dt = datetime.strptime(f"{date} {time}", "%Y-%m-%d %H:%M")
    end_dt = start_dt + timedelta(minutes=duration)

    cursor.execute("""
        SELECT start, end FROM events
        WHERE DATE(start) = ?
    """, (date,))
    slots = cursor.fetchall()
    conn.close()

    for slot in slots:
        # existing_start = datetime.strptime(f"{date} {slot[0]}", "%Y-%m-%d %H:%M:%S")
        # existing_end = datetime.strptime(f"{date} {slot[1]}", "%Y-%m-%d %H:%M:%S")
        existing_start = datetime.fromisoformat(slot[0]) 
        existing_end = datetime.fromisoformat(slot[1]) 
        if (start_dt < existing_end and end_dt > existing_start):
            return False
    return True

def suggest_alternate_slots(date, time, duration):
    base_dt = datetime.strptime(f"{date} {time}", "%Y-%m-%d %H:%M")
    suggestions = []
    deltas = [15, 30, 45, 60]

    for delta in deltas:
        new_time = base_dt + timedelta(minutes=delta)
        new_date = new_time.strftime("%Y-%m-%d")
        new_time_str = new_time.strftime("%H:%M")

        if is_slot_available(new_date, new_time_str, duration):
            suggestions.append(new_time_str)
            
        if len(suggestions) >= 2:
            break
    return suggestions

def extract_and_schedule(user_input: str):
    prompt = SCHEDULE_PROMPT.format(user_input=user_input)
    messages = [
        SystemMessage(content="Extract meeting details for scheduling."),
        HumanMessage(content=prompt)
    ]

    response = llm(messages)
    slots = clean_llm_response(response.content)
    if not slots:
        return {"error": "Could not extract meeting details."}

    available = is_slot_available(slots["date"], slots["time"], slots["duration_minutes"])

    if available:
        start_dt = datetime.strptime(f"{slots['date']} {slots['time']}", "%Y-%m-%d %H:%M")
        end_dt = start_dt + timedelta(minutes=slots["duration_minutes"])

        event_id = f"event-{slots['date'].replace('-', '')}{slots['time'].replace(':', '')}"

        event_type = slots.get('type', 'Meeting').capitalize()
        participants = slots.get('participant', [])
        event_title = f"{event_type} with {', '.join(participants)}"

        event_description = slots.get("purpose", "")

        event_start = start_dt.strftime("%Y-%m-%dT%H:%M:%S")
        event_end = end_dt.strftime("%Y-%m-%dT%H:%M:%S")

        event_color = "#2E86AB"

        event = {
            "id": event_id,
            "title": event_title,
            "description": event_description,
            "start": event_start,
            "end": event_end,
            "color": event_color,
            "participant": participants
        }

        ev_stat = insert_event(event=event)

        if str(ev_stat).lower() == 'true':

            return {
                "status": "available",
                "message": f"Slot is available for {slots['date']} at {slots['time']} and booked",
                # "slots": slots
                "event": event
            }
        else:
            return {
                "status": "available",
                "message": f"ERROR inserting into DB.Slot is available for {slots['date']} at {slots['time']}",
                # "slots": slots
                "event": {}
            }
    else:
        alternatives = suggest_alternate_slots(slots["date"], slots["time"], slots["duration_minutes"])
        return {
            "status": "unavailable",
            "message": f"Slot not available at {slots['time']} on {slots['date']}.",
            "suggestions": alternatives,
            "original_slots": slots
        }


In [12]:
if __name__ == "__main__":
    user_query = "I need to schedule a consultation with Dr. Smith on June 10th at 2 PM for 30 minutes."
    result = extract_and_schedule(user_query)
    print(json.dumps(result, indent=2))

{
  "status": "unavailable",
  "message": "Slot not available at 14:00 on 2025-06-10.",
  "suggestions": [
    "14:30",
    "14:45"
  ],
  "original_slots": {
    "participant": [
      "Dr. Smith"
    ],
    "date": "2025-06-10",
    "time": "14:00",
    "type": "consultation",
    "purpose": "Health check-up",
    "duration_minutes": 30
  }
}


In [6]:
fetch_all_events()

[{'id': 'event-202506101400',
  'title': 'Consultation with Dr. Smith',
  'description': 'Health check-up',
  'start': '2025-06-10T14:00:00',
  'end': '2025-06-10T14:30:00',
  'color': '#2E86AB',
  'participant': ['Dr. Smith']}]

# Chitchat

In [13]:
from langchain_ollama import ChatOllama
from langchain.schema import HumanMessage

llm = ChatOllama(model="gemma3:4b") 

def chitchat(user_input: str) -> str:
    
    chitchat_prompt = f"""
    Hi there! I'm your meet scheduling assistant. Feel free to ask me anything—or just chat! 
    By the way, I can also help you book or manage your appointments.
    
    Always say this helpful example to guide users: 
    'I want to book a meeting with Dr. Smith next Tuesday' whenever you're ready :)

    Respond politely and informally.

    User Input: {user_input}
    """
    response = llm([HumanMessage(content=chitchat_prompt.strip())])
    return response.content.strip()

In [14]:
if __name__ == "__main__":
    user_input = "How are you today?"
    reply = chitchat(user_input)
    print("Assistant:", reply)

Assistant: Hey there! I’m doing great, thanks for asking! Just here and ready to help you with scheduling – or just chat if you like! 😊 

I want to book a meeting with Dr. Smith next Tuesday whenever you’re ready :)


# Modifying

In [15]:
from langchain_ollama import ChatOllama
from langchain.schema import HumanMessage, SystemMessage
from datetime import datetime, timedelta
import sqlite3
import json
import re
from models.event import insert_event

llm = ChatOllama(model="gemma3:4b")

RESCHEDULE_PROMPT = """
You are an intelligent assistant that helps users reschedule appointments.

Extract these fields from the user message:
- participant: list of people involved
- original_date: in YYYY-MM-DD format
- original_time: in HH:MM (24-hour)
- new_date: in YYYY-MM-DD format
- new_time: in HH:MM (24-hour)
- duration_minutes: integer
- purpose: short description of the meeting purpose (optional)

Respond ONLY in JSON format:
{{
  "participant": ["Dr. Smith"],
  "original_date": "2025-06-10",
  "original_time": "14:00",
  "new_date": "2025-06-12",
  "new_time": "15:30",
  "duration_minutes": 30,
  "purpose": "Follow-up"
}}

User: {user_input}
"""

def clean_llm_response(raw_response):
    try:
        raw_response = re.sub(r"```json|```", "", raw_response).strip()
        match = re.search(r'\{.*?\}', raw_response, re.DOTALL)
        if match:
            return json.loads(match.group(0))
    except Exception as e:
        print("Error parsing JSON:", e)
    return None

def is_slot_available(date, time, duration):
    conn = sqlite3.connect("calendar_events.db")
    cursor = conn.cursor()
    start_dt = datetime.strptime(f"{date} {time}", "%Y-%m-%d %H:%M")
    end_dt = start_dt + timedelta(minutes=duration)

    cursor.execute("SELECT start, end FROM events WHERE DATE(start) = ?", (date,))
    slots = cursor.fetchall()
    conn.close()

    for slot in slots:
        existing_start = datetime.fromisoformat(slot[0])
        existing_end = datetime.fromisoformat(slot[1])
        if start_dt < existing_end and end_dt > existing_start:
            return False
    return True

def suggest_alternate_slots(date, time, duration):
    base_dt = datetime.strptime(f"{date} {time}", "%Y-%m-%d %H:%M")
    suggestions = []
    deltas = [15, 30, 45, 60]

    for delta in deltas:
        new_time = base_dt + timedelta(minutes=delta)
        new_date = new_time.strftime("%Y-%m-%d")
        new_time_str = new_time.strftime("%H:%M")

        if is_slot_available(new_date, new_time_str, duration):
            suggestions.append(new_time_str)
        if len(suggestions) >= 2:
            break
    return suggestions

def extract_and_reschedule(user_input: str):
    prompt = RESCHEDULE_PROMPT.format(user_input=user_input)
    messages = [
        SystemMessage(content="Extract rescheduling details."),
        HumanMessage(content=prompt)
    ]
    response = llm(messages)
    slots = clean_llm_response(response.content)

    if not slots:
        return {"error": "Could not extract rescheduling details."}

    available = is_slot_available(slots["new_date"], slots["new_time"], slots["duration_minutes"])
    
    if not available:
        alternatives = suggest_alternate_slots(slots["new_date"], slots["new_time"], slots["duration_minutes"])
        return {
            "status": "unavailable",
            "message": f"Slot not available at {slots['new_time']} on {slots['new_date']}.",
            "suggestions": alternatives,
            "slots": slots
        }

    # Remove old appointment
    conn = sqlite3.connect("calendar_events.db")
    cursor = conn.cursor()
    cursor.execute("""
        DELETE FROM events 
        WHERE DATE(start) = ? AND TIME(start) = ? AND participants LIKE ?
    """, (slots["original_date"], f"{slots['original_time']}:00", f"%{slots['participant'][0]}%"))
    conn.commit()

    # Prepare and insert new event
    start_dt = datetime.strptime(f"{slots['new_date']} {slots['new_time']}", "%Y-%m-%d %H:%M")
    end_dt = start_dt + timedelta(minutes=slots["duration_minutes"])
    event = {
        "id": f"event-{slots['new_date'].replace('-', '')}{slots['new_time'].replace(':', '')}",
        "title": f"Rescheduled with {', '.join(slots['participant'])}",
        "description": slots.get("purpose", ""),
        "start": start_dt.strftime("%Y-%m-%dT%H:%M:%S"),
        "end": end_dt.strftime("%Y-%m-%dT%H:%M:%S"),
        "color": "#F39C12",  
        "participant": slots["participant"]
    }

    insert_event(event)

    return {
        "status": "rescheduled",
        "message": f"Rescheduled to {slots['new_date']} at {slots['new_time']}.",
        "event": event
    }


In [17]:
if __name__ == "__main__":
    user_input = "Can you move my appointment with Dr. Smith from June 10 at 2 PM to June 12 at 3:30 PM?"
    result = extract_and_reschedule(user_input)
    print(json.dumps(result, indent=2))

{
  "status": "rescheduled",
  "message": "Rescheduled to 2025-06-12 at 15:30.",
  "event": {
    "id": "event-202506121530",
    "title": "Rescheduled with Dr. Smith",
    "description": "Appointment Reschedule",
    "start": "2025-06-12T15:30:00",
    "end": "2025-06-12T16:30:00",
    "color": "#F39C12",
    "participant": [
      "Dr. Smith"
    ]
  }
}


In [18]:
fetch_all_events()

[{'id': 'event-202506121530',
  'title': 'Rescheduled with Dr. Smith',
  'description': 'Appointment Reschedule',
  'start': '2025-06-12T15:30:00',
  'end': '2025-06-12T16:30:00',
  'color': '#F39C12',
  'participant': ['Dr. Smith']}]

# Email Confirmation

In [1]:
import sys
sys.path.append(r"C:\Users\unmes\Documents\RAGful_dev\meet_scheduler")
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import os
from config import SMTP_CONFIG


print("SMTP Host:", SMTP_CONFIG["host"])
print("SMTP Port:", SMTP_CONFIG["port"])
print("Sender Email:", SMTP_CONFIG["sender_email"])
print("Password Loaded:", "Yes" if SMTP_CONFIG["password"] else "No")

def send_confirmation_email(
    recipient_emails,
    meeting_type,
    meeting_date,
    meeting_start_time,
    meeting_end_time,
    participants,
    purpose
):
    subject = f"Meeting Confirmation - {meeting_type.capitalize()} with {', '.join(participants)}"
    
    body = f"""Dear Participant,

This is to confirm your scheduled meeting:

Type: {meeting_type.capitalize()}
Date: {meeting_date}
Time: {meeting_start_time} - {meeting_end_time}
Participants: {', '.join(participants)}
Purpose: {purpose}

If you have any questions or need to reschedule, please contact us.

Thank you,
Meet Schedular
"""

    msg = MIMEMultipart()
    msg['From'] = SMTP_CONFIG["sender_email"]
    msg['To'] = ', '.join(recipient_emails)
    msg['Subject'] = subject
    msg.attach(MIMEText(body, 'plain'))

    try:
        server = smtplib.SMTP(SMTP_CONFIG["host"], SMTP_CONFIG["port"])
        server.starttls()
        server.login(SMTP_CONFIG["sender_email"], SMTP_CONFIG["password"])
        server.sendmail(SMTP_CONFIG["sender_email"], recipient_emails, msg.as_string())
        server.quit()
        print("Email sent successfully!")
        return True
    except Exception as e:
        print(f"Failed to send email: {e}")
        return False

SMTP Host: smtp.gmail.com
SMTP Port: 587
Sender Email: meetschedular@gmail.com
Password Loaded: Yes


In [2]:
if __name__ == "__main__":
    recipient_emails = ["unmeshsupekar3@gmail.com"]
    send_confirmation_email(
        recipient_emails=recipient_emails,
        meeting_type="consultation",
        meeting_date="2025-06-10",
        meeting_start_time="14:00",
        meeting_end_time="14:30",
        participants=["Dr. Smith"],
        purpose="Health check-up"
    )

Email sent successfully!


# Calendar display

In [ ]:
import sys
sys.path.append(r"C:\Users\unmes\Documents\RAGful_dev\meet_scheduler")
import streamlit as st
from streamlit_calendar import calendar
import sqlite3
import json
from models.event import fetch_all_events

st.title("📅 Meeting Calendar")

event_list = fetch_all_events()

calendar_options = {
    "initialView": "timeGridWeek",
    "editable": False,
    "selectable": False,
    "headerToolbar": {
        "left": "prev,next today",
        "center": "title",
        "right": "dayGridMonth,timeGridWeek,timeGridDay"
    },
}

calendar(events=event_list, options=calendar_options)
